## Tarea 7 del curso CM-072

* Nombre y apellidos:
* Fecha de presentación: 17 de octubre.



LendingClub es una compañía de préstamos *peer-to-peer* que conecta directamente a los prestatarios con potenciales prestamistas/inversionistas.

Construirás un modelo de clasificación para predecir si un préstamo realizado a través del LendingClub tiene probabilidad de no ser pagado.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

loans = pd.read_csv("lending-club-data.csv", low_memory=False)


1 . Carga en una variable de nombre `todo_columnas` el nombre de todas las columnas del conjunto de datos.

In [2]:
# Solucion
todo_columnas=list(loans)
print(todo_columnas)

['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'not_compliant', 'status', 'inactive_loans', 'bad_loans', 'emp_length_num', 'grade_num', 'sub_grade_num', 'delinq_2yrs_zero', 'pub_rec_zero', 'collections_12_mths_zero', 'short_emp', 'payment_in

2 . La columna que contiene la información que queremos predecir se llama `malos_prestamos`. En esta columna, el valor 1 significa un préstamo riesgoso (malo), mientras que 0 significa un préstamos seguro.

Para hacer el trabajo más intuitivo, crea una nueva columna `prestamos_seguros` con el siguiente valor:

* +1 si es un préstamo seguro
* -1 si es un préstamos riesgoso (malo)

In [3]:
# Solucion
loans=loans.assign(prestamos_seguros=2*(loans['bad_loans']==0)-1)
print(loans['prestamos_seguros'].head())
print(len(loans))

0    1
1   -1
2    1
3    1
4    1
Name: prestamos_seguros, dtype: int64
122607


3 . Calcula la distribución en porcentaje de préstamos malos y préstamos buenos (debe sumar 100%).

In [4]:
# Tu solución
prestamos_totales=len(loans['prestamos_seguros'])
#print(prestamos_totales)
prestamos_buenos=((loans['prestamos_seguros']==1)*1).sum()
prestamos_malos=((loans['prestamos_seguros']==-1)*1).sum()

print("Porcentaje de prestamos Buenos: {0:.2f}".format(prestamos_buenos/prestamos_totales*100))
print()
print("Porcentaje de prestamos Malos: {0:.2f}".format(prestamos_malos/prestamos_totales*100))


#print(prestamos_buenos,prestamos_malos,(prestamos_buenos+prestamos_malos))
#print(loans['prestamos_seguros']==-1)

Porcentaje de prestamos Buenos: 81.12

Porcentaje de prestamos Malos: 18.88


4 . Una manera de abordar conjuntos de datos desbalanceados es con un submuestreo  de la clase más grande hasta que la distribución de clases sea mitad y mitad. Vamos a realizar un submuestreo de los préstamos buenos para balancear nuestro conjunto de datos. Ello significa que vamos a descartar muchas observaciones. 

* Pon en una variable `prestamos_arriesgado` todos y solo los préstamos malos.
* Pon en una variable `prestamos_seguros` una muestra aleatoria de préstamos buenos **del mismo tamaño** que la cantidad de préstamos malos. (Usa [pandas.DataFrame.sample](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html) con el atributo `random_state=0`)
* Junta en una nueva variable `prestamos_balanceados`, los dos grupos anteriores: `prestamos_arriesgados` y `prestamos_seguros`.

In [5]:
# Tus soluciones
#prestamos_seguros=loans.loc["prestamos_seguros"==1].sample(len(prestamos_malos))
prestamos_seguros=loans.loc[loans["prestamos_seguros"]==1].sample(prestamos_malos)
prestamos_arriesgado=loans.loc[loans["prestamos_seguros"]==-1]


prestamos_balanceados=pd.concat([prestamos_seguros,prestamos_arriesgado])

print(prestamos_balanceados.head(1))
print(len(prestamos_balanceados))

            id  member_id  loan_amnt  funded_amnt  funded_amnt_inv  \
63114  3932766    5156804      12000        12000            12000   

             term  int_rate  installment grade sub_grade        ...         \
63114   36 months     10.16       388.11     B        B1        ...          

      delinq_2yrs_zero pub_rec_zero collections_12_mths_zero  short_emp  \
63114              1.0          1.0                      1.0          0   

      payment_inc_ratio          final_d last_delinq_none last_record_none  \
63114           6.33649  20160501T000000                1                1   

      last_major_derog_none prestamos_seguros  
63114                     1                 1  

[1 rows x 69 columns]
46300


5 . Asigna a una variable `subconjunto_prestamos` sólo el siguiente subconjunto de características que son las que usaremos:

```python
caracteristica = ['grade',               # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]
```

Asimismo, asigna a una variable **`y`** los valores de la columna `prestamos_seguros`.

In [6]:
caracteristica = ['grade',               # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]
subconjunto_prestamos=prestamos_balanceados[caracteristica]
print(subconjunto_prestamos.head())


y = prestamos_balanceados[['prestamos_seguros']]
print(y.head())

      grade sub_grade  short_emp  emp_length_num home_ownership    dti  \
63114     B        B1          0              11           RENT  20.84   
87510     C        C1          0               9           RENT  14.74   
45305     B        B5          0               8       MORTGAGE  19.26   
12467     C        C4          1               0       MORTGAGE  18.11   
51207     D        D3          0              11           RENT  16.70   

                  purpose        term  last_delinq_none  \
63114  debt_consolidation   36 months                 1   
87510  debt_consolidation   36 months                 0   
45305         credit_card   36 months                 1   
12467  debt_consolidation   36 months                 0   
51207             medical   36 months                 1   

       last_major_derog_none  revol_util  total_rec_late_fee  
63114                      1        82.1                 0.0  
87510                      1        55.7                 0.0  
45305      

6 .  Usando [pandas.get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) convierte las variables categóricas de `subconjunto_prestamos` en variables numéricas *one-hot*. Guarda el nuevo conjunto de datos en `X`.

In [7]:
# Tu solucion

print(list(subconjunto_prestamos))
X=pd.get_dummies(subconjunto_prestamos)
print("\n\n")
print(list(X))

['grade', 'sub_grade', 'short_emp', 'emp_length_num', 'home_ownership', 'dti', 'purpose', 'term', 'last_delinq_none', 'last_major_derog_none', 'revol_util', 'total_rec_late_fee']



['short_emp', 'emp_length_num', 'dti', 'last_delinq_none', 'last_major_derog_none', 'revol_util', 'total_rec_late_fee', 'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G', 'sub_grade_A1', 'sub_grade_A2', 'sub_grade_A3', 'sub_grade_A4', 'sub_grade_A5', 'sub_grade_B1', 'sub_grade_B2', 'sub_grade_B3', 'sub_grade_B4', 'sub_grade_B5', 'sub_grade_C1', 'sub_grade_C2', 'sub_grade_C3', 'sub_grade_C4', 'sub_grade_C5', 'sub_grade_D1', 'sub_grade_D2', 'sub_grade_D3', 'sub_grade_D4', 'sub_grade_D5', 'sub_grade_E1', 'sub_grade_E2', 'sub_grade_E3', 'sub_grade_E4', 'sub_grade_E5', 'sub_grade_F1', 'sub_grade_F2', 'sub_grade_F3', 'sub_grade_F4', 'sub_grade_F5', 'sub_grade_G1', 'sub_grade_G2', 'sub_grade_G3', 'sub_grade_G4', 'sub_grade_G5', 'home_ownership_MORTGAGE', 'home_ownership_OTHER', 'home_owne

7 .  Empleando `sklearn.model_selection.train_test_split` separa el conjunto de datos en un 90% para entrenamiento y validación (`X_entrenamiento_val`, `y_entrenamiento_val`), y 10% para pruebas (`X_prueba`, `y_prueba`).

Luego separa (`X_entrenamiento_val`, `y_entrenamiento_val`) en un 80% para entrenamiento (`X_entrenamiento`, `y_entrenamiento`) y 20% para validación (`X_val`, `y_val`).

In [8]:
# Tu solucion
from sklearn.model_selection import train_test_split


X_entrenamiento_val, X_prueba, y_entrenamiento_val, y_prueba = train_test_split(X, y, test_size=0.10, random_state=0)
print(len(X_entrenamiento_val),len(X_prueba))

X_entrenamiento_val, X_prueba, y_entrenamiento_val, y_prueba = train_test_split(X, y, test_size=0.20, random_state=0)
print(len(X_entrenamiento_val),len(X_prueba))

41670 4630
37040 9260


8 . Entrena un modelo como Regresión Logística, Naive Bayes, KNN y un cuarto modelo de tu elección, con las siguientes indicaciones:

* Utilizar el uso apropiado de la normalización (Scaling) de datos si fuese necesario.
* El uso apropiado de una técnica para la selección de los mejores parámetros de cada modelo (p.ej. búsqueda grid o búsqueda aleatoria)
* Reporte para cada modelo la exactitud , precisión y exhaustividad, F1-Score  **en el conjunto de pruebas.** y muestra la matriz de confusión.
* Comenta tus resultados.

In [9]:
# Tus soluciones

#Scaling

from sklearn.preprocessing import StandardScaler

sc=StandardScaler()

X_entrenamiento_val=sc.fit_transform(X_entrenamiento_val)
X_prueba = sc.transform(X_prueba)




In [10]:
#Regresión Logísta

from sklearn.linear_model import LogisticRegression

#y_entremiento_lr=y_entrenamiento_val.as_matrix
#print(list(X_entrenamiento_val))
#print(X_entrenamiento_val)
LR= LogisticRegression().fit(X_entrenamiento_val,(y_entrenamiento_val.values).ravel())
#print("ok")
from sklearn import metrics
predicciones_test_LR = LR.predict(X_prueba)
matriz_confusion_LR = metrics.confusion_matrix(y_prueba, predicciones_test_LR)
TN_LR = matriz_confusion_LR[0,0]
FN_LR = matriz_confusion_LR[1,0]
FP_LR = matriz_confusion_LR[0,1]
TP_LR = matriz_confusion_LR[1,1]

print ('              +-----------------+')
print ('              |   Predicción    |')
print ('              +-----------------+')
print ('              |    +   |    -   |')
print ('+-------+-----+--------+--------+')
print ('| Valor |  +  |   %d |  %d  |'   % (TP_LR, FN_LR) )
print ('| real  +-----+--------+--------+')
print ('|       |  -  | %d   | %d   |'    % (FP_LR, TN_LR) )
print ('+-------+-----+--------+--------+')
print ()
#print ( 'Exactitud    : ', (TP_LR+TN_LR)/(TP_LR+FN_LR+FP_LR+TN_LR) )
print('Exactitud     :',metrics.accuracy_score(y_prueba, predicciones_test_LR) )
print('Precisión     :',metrics.precision_score(y_prueba, predicciones_test_LR)) 
print('Exhaustividad :',metrics.recall_score(y_prueba, predicciones_test_LR))
print('F1-Score      :',metrics.f1_score(y_prueba, predicciones_test_LR))

              +-----------------+
              |   Predicción    |
              +-----------------+
              |    +   |    -   |
+-------+-----+--------+--------+
| Valor |  +  |   2935 |  1663  |
| real  +-----+--------+--------+
|       |  -  | 1647   | 3015   |
+-------+-----+--------+--------+

Exactitud     : 0.642548596112311
Precisión     : 0.6405499781754692
Exhaustividad : 0.6383210091344063
F1-Score      : 0.639433551198257


In [11]:
from sklearn.naive_bayes import GaussianNB

GNB = GaussianNB().fit(X_entrenamiento_val,(y_entrenamiento_val.values).ravel())

predicciones_test_GNB=GNB.predict(X_prueba)

matriz_confusion_GNB = metrics.confusion_matrix(y_prueba, predicciones_test_GNB)
TN_GNB = matriz_confusion_GNB[0,0]
FN_GNB = matriz_confusion_GNB[1,0]
FP_GNB = matriz_confusion_GNB[0,1]
TP_GNB = matriz_confusion_GNB[1,1]

print ('              +-----------------+')
print ('              |   Predicción    |')
print ('              +-----------------+')
print ('              |    +   |    -   |')
print ('+-------+-----+--------+--------+')
print ('| Valor |  +  |   %d |  %d  |'   % (TP_GNB, FN_GNB) )
print ('| real  +-----+--------+--------+')
print ('|       |  -  | %d   | %d   |'    % (FP_GNB, TN_GNB) )
print ('+-------+-----+--------+--------+')
print ()
#print ('Exactitud    : ', (TP_GNB+TN_GNB)/(TP_GNB+FN_GNB+FP_GNB+TN_GNB) )
print('Exactitud     :',metrics.accuracy_score(y_prueba, predicciones_test_GNB) )
print('Precisión     :',metrics.precision_score(y_prueba, predicciones_test_GNB)) 
print('Exhaustividad :',metrics.recall_score(y_prueba, predicciones_test_GNB))
print('F1-Score      :',metrics.f1_score(y_prueba, predicciones_test_GNB))


              +-----------------+
              |   Predicción    |
              +-----------------+
              |    +   |    -   |
+-------+-----+--------+--------+
| Valor |  +  |   3724 |  874  |
| real  +-----+--------+--------+
|       |  -  | 2805   | 1857   |
+-------+-----+--------+--------+

Exactitud     : 0.6026997840172786
Precisión     : 0.570378312145811
Exhaustividad : 0.8099173553719008
F1-Score      : 0.6693628111800126


In [12]:
from sklearn.neighbors import KNeighborsClassifier


KNN= KNeighborsClassifier().fit(X_entrenamiento_val,(y_entrenamiento_val.values).ravel())

predicciones_test_KNN=KNN.predict(X_prueba)

matriz_confusion_KNN = metrics.confusion_matrix(y_prueba, predicciones_test_KNN)
TN_KNN = matriz_confusion_KNN[0,0]
FN_KNN = matriz_confusion_KNN[1,0]
FP_KNN = matriz_confusion_KNN[0,1]
TP_KNN = matriz_confusion_KNN[1,1]

print ('              +-----------------+')
print ('              |   Predicción    |')
print ('              +-----------------+')
print ('              |    +   |    -   |')
print ('+-------+-----+--------+--------+')
print ('| Valor |  +  |   %d |  %d  |'   % (TP_KNN, FN_KNN) )
print ('| real  +-----+--------+--------+')
print ('|       |  -  | %d   | %d   |'    % (FP_KNN, TN_KNN) )
print ('+-------+-----+--------+--------+')
print ()
#print ('Exactitud    : ', (TP_GNB+TN_GNB)/(TP_GNB+FN_GNB+FP_GNB+TN_GNB) )
print('Exactitud     :',metrics.accuracy_score(y_prueba, predicciones_test_KNN) )
print('Precisión     :',metrics.precision_score(y_prueba, predicciones_test_KNN)) 
print('Exhaustividad :',metrics.recall_score(y_prueba, predicciones_test_KNN))
print('F1-Score      :',metrics.f1_score(y_prueba, predicciones_test_KNN))


              +-----------------+
              |   Predicción    |
              +-----------------+
              |    +   |    -   |
+-------+-----+--------+--------+
| Valor |  +  |   2721 |  1877  |
| real  +-----+--------+--------+
|       |  -  | 1925   | 2737   |
+-------+-----+--------+--------+

Exactitud     : 0.5894168466522678
Precisión     : 0.5856650882479553
Exhaustividad : 0.5917790343627665
F1-Score      : 0.5887061877974904


In [14]:
from sklearn.svm import SVC

Clasificador_SVC=SVC(kernel='linear', random_state=0).fit(X_entrenamiento_val,(y_entrenamiento_val.values).ravel())

In [15]:
predicciones_test_SVC=Clasificador_SVC.predict(X_prueba)

matriz_confusion_SVC = metrics.confusion_matrix(y_prueba, predicciones_test_SVC)
TN_SVC = matriz_confusion_SVC[0,0]
FN_SVC = matriz_confusion_SVC[1,0]
FP_SVC = matriz_confusion_SVC[0,1]
TP_SVC = matriz_confusion_SVC[1,1]

print ('              +-----------------+')
print ('              |   Predicción    |')
print ('              +-----------------+')
print ('              |    +   |    -   |')
print ('+-------+-----+--------+--------+')
print ('| Valor |  +  |   %d |  %d  |'   % (TP_SVC, FN_SVC) )
print ('| real  +-----+--------+--------+')
print ('|       |  -  | %d   | %d   |'    % (FP_SVC, TN_SVC) )
print ('+-------+-----+--------+--------+')
print ()
#print ('Exactitud    : ', (TP_GNB+TN_GNB)/(TP_GNB+FN_GNB+FP_GNB+TN_GNB) )
print('Exactitud     :',metrics.accuracy_score(y_prueba, predicciones_test_SVC) )
print('Precisión     :',metrics.precision_score(y_prueba, predicciones_test_SVC)) 
print('Exhaustividad :',metrics.recall_score(y_prueba, predicciones_test_SVC))
print('F1-Score      :',metrics.f1_score(y_prueba, predicciones_test_SVC))

              +-----------------+
              |   Predicción    |
              +-----------------+
              |    +   |    -   |
+-------+-----+--------+--------+
| Valor |  +  |   2906 |  1692  |
| real  +-----+--------+--------+
|       |  -  | 1785   | 2877   |
+-------+-----+--------+--------+

Exactitud     : 0.6245140388768898
Precisión     : 0.6194841185248348
Exhaustividad : 0.6320139190952588
F1-Score      : 0.6256862956184736
